In [1]:
from keras.applications.vgg16 import VGG16, preprocess_input
# from keras.applications.vgg19 import VGG19, preprocess_input
from keras.applications.xception import Xception, preprocess_input
# from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras import applications
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

Using TensorFlow backend.


In [2]:
img_width, img_height = 224,224

In [3]:
model = applications.InceptionV3(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))

In [14]:
inceptionv3=Model(input=model.input, output=model.get_layer('mixed10').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("mi..., inputs=Tensor("in...)`
  """Entry point for launching an IPython kernel.


In [5]:
from keras.models import model_from_json

In [7]:
import os

In [8]:
os.mkdir("model_json")

In [15]:
# serialize model to JSON
model_json = inceptionv3.to_json()
with open("model_json/inceptionv3.json", "w") as json_file:
    json_file.write(model_json)

In [10]:
### For fine tune
# Top Model Block
'''
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(nb_classes, activation='softmax')(x)

    # add your top layer block to your base model
    model = Model(base_model.input, predictions)
    print(model.summary())

    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train
    # # uncomment when choosing based_model_last_block_layer
    # for i, layer in enumerate(model.layers):
    #     print(i, layer.name)

    # first: train only the top layers (which were randomly initialized)
    # i.e. freeze all layers of the based model that is already pre-trained.
    for layer in base_model.layers:
        layer.trainable = False
'''

"\n    x = base_model.output\n    x = GlobalAveragePooling2D()(x)\n    predictions = Dense(nb_classes, activation='softmax')(x)\n\n    # add your top layer block to your base model\n    model = Model(base_model.input, predictions)\n    print(model.summary())\n\n    # # let's visualize layer names and layer indices to see how many layers/blocks to re-train\n    # # uncomment when choosing based_model_last_block_layer\n    # for i, layer in enumerate(model.layers):\n    #     print(i, layer.name)\n\n    # first: train only the top layers (which were randomly initialized)\n    # i.e. freeze all layers of the based model that is already pre-trained.\n    for layer in base_model.layers:\n        layer.trainable = False\n"

In [12]:
model_ = applications.VGG16(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
vgg=Model(input=model_.input, output=model_.get_layer('block5_pool').output)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [13]:
# serialize model to JSON
model_json = vgg.to_json()
with open("model_json/vgg16.json", "w") as json_file:
    json_file.write(model_json)

In [17]:
xception =applications.Xception(weights='imagenet',include_top=False, input_shape = (img_width, img_height, 3))
feat_ext_xception=Model(input=xception.input, output=xception.get_layer("block14_sepconv2_act").output)
# serialize model to JSON
model_json = feat_ext_xception.to_json()
with open("model_json/xception.json", "w") as json_file:
    json_file.write(model_json)

/Users/rishikesh/anaconda/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:2: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("bl..., inputs=Tensor("in...)`
  


In [18]:
## After save the models

In [3]:
from keras.preprocessing import image
from keras.models import Model
from keras.models import model_from_json

# other imports
from sklearn.preprocessing import LabelEncoder
import numpy as np
import glob
import h5py
import os
import json
import pickle as cPickle
import datetime
import time

In [4]:
img_width, img_height = 224,224

In [5]:
img_size=224

In [6]:
train_data_dir = "data/MSTAR_Chips/TRAIN_images"
validation_data_dir = "data/MSTAR_Chips/TEST_images"
nb_train_samples = 1622
nb_validation_samples = 1365 
batch_size = 32
epochs = 50

In [7]:
# path to training dataset
train_labels = os.listdir(train_data_dir)

In [13]:
import os
import glob

models=[]
model_names=[]
json_dir_name = "model_json"
weight_name=""
json_pattern = os.path.join(json_dir_name,'*.json')
file_list = glob.glob(json_pattern)
for file in file_list:
    print("Json file path :",file)
    # load json and create model
    json_file = open(file, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    weight_name= file.split(".")
    print("Weight path : {}.h5".format(weight_name[0]))
    #load weights into new model
    loaded_model.load_weights("{}.h5".format(weight_name[0]))
    models.append(loaded_model)  
    model_names.append(weight_name[0].split("\\")[-1])

Json file path : model_json\inceptionv3.json
Weight path : model_json\inceptionv3.h5
Json file path : model_json\vgg16.json
Weight path : model_json\vgg16.h5
Json file path : model_json\xception.json
Weight path : model_json\xception.h5


In [14]:
model_names

['inceptionv3', 'vgg16', 'xception']

In [9]:
import h5py

In [19]:
image_size=(224,224)

with open("label.pkl", "wb") as label_file:
    for model,name in zip(models,model_names):
        features = []
        labels = []
        # h5f = h5py.File('features/{}.h5'.format(name), 'w')
        with h5py.File('features/{}.hdf5'.format(name),'w') as f:
            # loop over all the labels in the folder
            for label in train_labels:
                cur_path = train_data_dir + "/" + label
                print(cur_path)
                for image_path in glob.glob(cur_path + "/*.jpg"):
                    img = image.load_img(image_path, target_size=image_size)
                    x = image.img_to_array(img)
                    x = np.expand_dims(x, axis=0)
                    x = preprocess_input(x)
                    feature = model.predict(x)
                    flat = feature.flatten()
                    features.append(flat)
                    labels.append(label)
            f.create_dataset('train', data=features)
    cPickle.dump(labels, label_file)

with open("test_label.pkl", "wb") as label_file:
    for model,name in zip(models,model_names):
        # h5f = h5py.File('features/{}.h5'.format(name), 'w')
        with h5py.File('features/{}.h5'.format(name),'w') as f:
            test_features = []
            test_labels = []
            # loop over all the labels in the folder
            for label in train_labels:
                cur_path = validation_data_dir + "/" + label
                for image_path in glob.glob(cur_path + "/*.jpg"):
                    img = image.load_img(image_path, target_size=image_size)
                    x = image.img_to_array(img)
                    x = np.expand_dims(x, axis=0)
                    x = preprocess_input(x) 
                    feature = model.predict(x)
                    flat = feature.flatten()
                    test_features.append(flat)
                    test_labels.append(label)
            f.create_dataset('test', data=test_features)
            
    cPickle.dump(test_labels, label_file)
#h5f.close()

data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank
data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank
data/MSTAR_Chips/TRAIN_images/bmp2_tank
data/MSTAR_Chips/TRAIN_images/btr70_transport
data/MSTAR_Chips/TRAIN_images/t72_tank


In [20]:
# For prediction work here

In [23]:
with open('test_label.pkl', 'rb') as f:
    test_labels = cPickle.load(f)
    
with open('label.pkl', 'rb') as f:
    labels = cPickle.load(f)

In [24]:
# encode the labels using LabelEncoder
targetNames = np.unique(labels)
le = LabelEncoder()
le_labels = le.fit_transform(labels)
test_labels=le.transform(test_labels)

In [25]:
feat_pattern = os.path.join("features/",'*.hdf5')
file_list = glob.glob(feat_pattern)
feat=[]
for file in file_list:
    print("Json file path :",file)
    # load json and create model
    with  h5py.File(file, 'r') as f:
        feat.append(f['train'][:])       

Json file path : features\inceptionv3.hdf5
Json file path : features\vgg16.hdf5
Json file path : features\xception.hdf5


In [29]:
i=0
for feature in feat:
    if i==0:
        features=np.asarray(feature)
    else:
        features = np.concatenate( [features,np.asarray(feature)], axis=1 )
    print("Feat: ",np.asarray(feature).shape)
    print("Features: ",features.shape)
    i=i+1

Feat:  (1622, 51200)
Features:  (1622, 51200)
Feat:  (1622, 25088)
Features:  (1622, 76288)
Feat:  (1622, 100352)
Features:  (1622, 176640)


In [37]:
test_feat_pattern = os.path.join("features/",'*.h5')
file_list = glob.glob(test_feat_pattern)
test_feat=[]
for file in file_list:
    print("Json file path :",file)
    # load json and create model
    with  h5py.File(file, 'r') as f:
        test_feat.append(f['test'][:]) 
        
i=0
for feature in test_feat:
    if i==0:
        test_features=np.asarray(feature)
    else:
        test_features = np.concatenate( [test_features,np.asarray(feature)], axis=1 )
    print("Feat: ",np.asarray(feature).shape)
    print("Features: ",test_features.shape)
    i=i+1

Json file path : features\inceptionv3.h5
Json file path : features\vgg16.h5
Json file path : features\xception.h5
Feat:  (1365, 51200)
Features:  (1365, 51200)
Feat:  (1365, 25088)
Features:  (1365, 76288)
Feat:  (1365, 100352)
Features:  (1365, 176640)


In [33]:
seed=2017

In [34]:
from sklearn.metrics import accuracy_score
# organize imports
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix

In [35]:
# use logistic regression as the model
print("[INFO] creating model...")
model = LogisticRegression(random_state=seed)
model.fit(features, le_labels)

[INFO] creating model...


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=2017, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
# evaluate the model of test data
preds = model.predict(test_features)
print(classification_report(le.inverse_transform(preds),le.inverse_transform(test_labels)))
print("Accuracy :",accuracy_score(le.inverse_transform(preds),le.inverse_transform(test_labels)))

                 precision    recall  f1-score   support

      bmp2_tank       0.87      0.92      0.90       555
btr70_transport       0.79      0.85      0.82       181
       t72_tank       0.97      0.90      0.94       629

    avg / total       0.91      0.90      0.91      1365

Accuracy : 0.904029304029
